In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/sales_train.csv")
test = pd.read_csv("../input/test.csv")
sample_submit = pd.read_csv("../input/sample_submission.csv")

**Description**

In [ ]:
print("the train_shape: ",train.shape);
print("the submission shape",sample_submit.shape )
sample_submit.head()

In [ ]:
#colums of sample_submit
print("colums of sample_submit:", sample_submit.columns)
#colums of train
print("colums of train:", train.columns)

***Cleanning data***

In [ ]:
#drop duplicate row
train = train.drop_duplicates()
# check if any value in a Series is Missing
if train.isnull().values.any():
    print("ha")
    train = train.dropna()
#remove date and item_price
train = train.drop(labels = ['date', 'item_price'], axis = 1)
#need to create item_cnt_month  in train 
train = train.groupby(['date_block_num','shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()
#rename item_cnt_day into item_cnt_month
train = train.rename(index=str, columns = {"item_cnt_day":"item_cnt_month"})

In [ ]:
#change columns
train = train.pivot_table(index=['item_id','shop_id'], columns='date_block_num',values='item_cnt_month',fill_value=0).reset_index()
#database = fusion(test et train)
database = pd.merge(test,train,on=['item_id','shop_id'], how='left').fillna(0)

In [ ]:
#just to see database 
database.head()

In [ ]:
train = database.iloc[:,3:37].values


In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
train_scaled = sc.fit_transform(train)


In [ ]:
#X_train,Y_train
Y_train = train_scaled[:,33]
X_train = train_scaled[:,0:33]
#X_test
X_test = train_scaled[:,1:]
print("Y_train shape:",Y_train.shape)
print("X_train shape:",X_train.shape)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
Y_train = np.reshape(Y_train, (Y_train.shape[0], 1))
print("Y_train re-shape:",Y_train.shape)
print("X_train re-shape:",X_train.shape)
print("X_test re-shape:",X_test.shape)

**# Building the RNN**

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
#model layer
model = Sequential()
model.add(LSTM(16, input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(optimizer="adam", loss='mse', metrics=["mse"])
print(model.summary())

In [ ]:
history = model.fit(X_train, Y_train, batch_size=200, epochs=10)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label= 'loss(mse)')
plt.plot(np.sqrt(history.history['mean_squared_error']), label= 'rmse')
plt.legend(loc=1)


**Prediction**

In [ ]:
Y_prediction = model.predict(X_test).clip(0., 20.)
prediction = pd.DataFrame(Y_prediction , columns=['item_cnt_month'])
prediction.to_csv('submission.csv',index_label='ID')